<a href="https://colab.research.google.com/github/Siddarth-Velan/IEEE-ML-Round-2/blob/main/IEEEML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beir sentence-transformers faiss-gpu-cu12 rank-bm25 ranx

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 74.4 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=w

In [2]:
import time
import pickle


In [3]:
import torch
print(torch.version.cuda)

12.6


In [4]:
import numpy as np
import pandas as pd
import sklearn
import random
from sentence_transformers import SentenceTransformer

In [5]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

In [6]:
dataset = "scidocs"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip"
data_path = util.download_and_unzip(url, ".")

corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="dev")

./msmarco.zip:   0%|          | 0.00/1.01G [00:00<?, ?iB/s]

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
idxs = list(corpus.keys())
random.seed(67)
data_idxs = random.sample(idxs, 100000)
data = {idx: corpus[idx] for idx in data_idxs}
text = [data[idx]['text'] for idx in data_idxs]
text2 = text3 = text
qid = list(queries.keys())
qrid = list(qrels.keys())
q_train, q_test, qr_train, qr_test = train_test_split(qid, qrid, test_size=0.2, random_state=67)

train_queries = {idx : queries[idx] for idx in q_train}
test_queries = {idx : queries[idx] for idx in q_test}
train_qrels = {idx : qrels[idx] for idx in qr_train if idx in qrels}
test_qrels = {idx : qrels[idx] for idx in qr_test if idx in qrels}

In [9]:
print(corpus.get(8762818))

None


In [10]:
print(len(train_queries), len(test_queries))

5584 1396


In [11]:
print(len(qrels))

6980


In [12]:
print(len(text))

100000


In [13]:
print(len(list(text)[0:10]))
print(text2[0][:2000])

10
This tells you how competitive the school is and how serious their requirements are. The acceptance rate at NYU is 35%. For every 100 applicants, 35 are admitted. This means the school is very selective. If you meet NYU's requirements for GPA, SAT/ACT scores, and other components of the application, you have a great shot at getting in.


In [14]:
with open('data_idxs.pkl', 'wb') as f:
    pickle.dump(data_idxs, f)

with open('texts.pkl', 'wb') as f:
    pickle.dump(text2, f)

In [15]:
baseline = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
components = baseline.encode(text, batch_size = 100, convert_to_numpy = True)
np.save('/content/drive/MyDrive/embeddings.npy', components)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
import faiss
embeds = np.load('/content/drive/MyDrive/embeddings.npy')
print(embeds.shape)
faiss_idx = faiss.IndexFlatIP(embeds.shape[1])
faiss.normalize_L2(embeds)
faiss_idx.add(embeds)
faiss.write_index(faiss_idx, 'faiss_idx_saved.index')

(100000, 384)


In [17]:
def search(query, k):
  query_embedding = baseline.encode([query], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = faiss_idx.search(query_embedding, k)
  scores = s[0]
  indices = indices[0]

  ranx_arr = []
  for i in range(len(indices)):
    id = data_idxs[indices[i]]
    score = float(scores[i])
    ranx_arr.append({'id' : id, 'score' : score})
  return ranx_arr

search("virus", 5)

[{'id': '3003879', 'score': 0.6138339042663574},
 {'id': '2461086', 'score': 0.5530322194099426},
 {'id': '8434758', 'score': 0.5415890216827393},
 {'id': '1087728', 'score': 0.5414258241653442},
 {'id': '5205661', 'score': 0.5411513447761536}]

In [18]:
import re
def tokenize(content):
  content = content.lower()
  tokens = re.findall(r'\w+', content)
  return tokens


In [19]:
from rank_bm25 import BM25Okapi
bm_25 = BM25Okapi([tokenize(i) for i in text])


In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [72]:
def hybrid_search(query, k_b, k_f, alpha):
  bm_score = bm_25.get_scores(tokenize(query))
  bm_idx = np.argsort(bm_score)[::-1][:k_b]
  bm_score = scaler.fit_transform(bm_score.reshape(-1, 1))
  bm = {}
  for idx in bm_idx:
    id = text3[idx]
    bm[id] = bm_score[idx][0]
  query_embedding = baseline.encode([query], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = faiss_idx.search(query_embedding, k_f)
  s = scaler.fit_transform(s.reshape(-1, 1))
  faiss_r = {}
  for score_array, idx in zip(s, indices[0]):
      id = text3[idx]
      faiss_r[id] = float(score_array[0])
  indices = indices[0]

  total = set(bm.keys()) | set(faiss_r.keys())
  answer = []
  for id in total:
    bm_s = bm.get(id, 0)
    faiss_s = faiss_r.get(id, 0)
    score = alpha * (bm_s) + (1-alpha)*(faiss_s)
    answer.append({'id': id , 'score' : score})

  return answer[:min(k_b, k_f)]

In [73]:
hybrid_search('viral', 5, 5, 0.5)

[{'id': 'The prognosis for viral meningitis is much better than that for bacterial meningitis, with most people recovering completely with simple treatment of the symptoms. Because antibiotics do not help viral infections, they are not useful in the treatment of viral meningitis.',
  'score': np.float64(0.5)},
 {'id': 'An RNA virus is a virus that has RNA (ribonucleic acid) as its genetic material.This nucleic acid is usually single-stranded RNA (ssRNA) but may be double-stranded RNA (dsRNA).Notable human diseases caused by RNA viruses include Ebola hemorrhoragic fever, SARS, influenza, hepatitis C, West Nile fever, polio, and measles.his nucleic acid is usually single-stranded RNA (ssRNA) but may be double-stranded RNA (dsRNA). Notable human diseases caused by RNA viruses include Ebola hemorrhoragic fever, SARS, influenza, hepatitis C, West Nile fever, polio, and measles.',
  'score': 0.21714067459106445},
 {'id': 'There are three main ways a virus can spread: person to person, enviro

In [74]:
hybrid_search('ebola', 1, 5, 0.8)

[{'id': 'The symptoms of the Ebola thus begin to show up. The period between the Ebola virus entering the body and the symptoms showing up is known as the incubation period. This incubation period of Ebola virus can be as short as 2 days or as long as 21 days.In some cases the symptoms of the Ebola virus can be seen from the fourth or the sixth day itself.or instance For example, the incubation period for Lake Victoria Marburg virusor Zaire Ebola viru s infection in rhesus macaques is between 3 to 16 days; however, for that of guinea pigs it is around 4 to 10 days. More Articles :',
  'score': np.float64(0.9999999523162841)}]

In [24]:
print(len(queries))

6980


In [25]:
print(len(qrels))

6980


In [26]:
from ranx import Qrels, Run, evaluate

In [27]:
ranx_qrels = Qrels(qrels)
run = {}
for id, text in queries.items():
  results = search(text,10)
  run[id] = {i['id']:i['score'] for i in results}
ranx_run = Run(run)
metrics = evaluate(ranx_qrels, ranx_run, ['mrr', 'ndcg@1', 'ndcg@5', 'ndcg@10', 'ndcg@20', 'recall@1', 'recall@5', 'recall@10', 'recall@100', 'precision@1', 'precision@5', 'precision@10'])

/usr/local/lib/python3.12/dist-packages/ranx/metrics/reciprocal_rank.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _reciprocal_rank(qrels[i], run[i], k, rel_lvl)


In [28]:
for metric, score in metrics.items():
    print(metric, score)

mrr 0.01004178605539637
ndcg@1 0.009025787965616046
ndcg@5 0.009789751592870839
ndcg@10 0.009876360432857029
ndcg@20 0.009876360432857029
recall@1 0.008357211079274117
recall@5 0.010506208213944605
recall@10 0.010792741165234003
recall@100 0.010792741165234003
precision@1 0.009025787965616046
precision@5 0.0022349570200573066
precision@10 0.001146131805157593


In [29]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import InputExample

In [30]:
training = []


In [31]:
for id in list(train_queries.keys()):
    query_text = train_queries[id]
    if id not in train_qrels:
        continue
    positive_id = list(train_qrels[id].keys())[0]
    positive_text = corpus[positive_id]['text']

    hard_neg = search(query_text, 5)
    negative_id = None
    for i in hard_neg:
      if i['id'] not in train_qrels.get(id):
        negative_id = i['id']
        break
    if negative_id is None:
      continue
    negative_text = corpus[negative_id]['text']
    training.append(InputExample(texts = (query_text, positive_text, negative_text)))

In [32]:
print(training)

[<sentence_transformers.readers.InputExample.InputExample object at 0x7b0e8f1abd40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e9e4fb890>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e8eb65ee0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0eda964950>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e8f07d700>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0eb40a2b10>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0ffd3b0d10>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e325344a0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e322357c0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e3191c3e0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0fde8fc920>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b0e31

In [33]:
from torch.utils.data import DataLoader

In [34]:
model2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [35]:
dataloader = DataLoader(training, batch_size=16)
loss = MultipleNegativesRankingLoss(model2)
model2.fit(train_objectives=[(dataloader, loss)], epochs = 3, output_path = '/content/drive/MyDrive/finetuned')


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.573917
1000,0.527268


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
final = SentenceTransformer('/content/drive/MyDrive/finetuned')

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

In [37]:
len(text2)

100000

In [38]:
embeddings_finetuned = final.encode(text2, batch_size=32, convert_to_numpy=True)
np.save('/content/drive/MyDrive/embeddings_finetuned.npy', embeddings_finetuned)
print(embeddings_finetuned.shape)
faiss.normalize_L2(embeddings_finetuned)
index_finetuned = faiss.IndexFlatIP(embeddings_finetuned.shape[1])
index_finetuned.add(embeddings_finetuned)
faiss.write_index(index_finetuned, 'finetuned_idx.index')

(100000, 384)


In [39]:
def search_test(query, k):
  query_embedding = final.encode([query], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = index_finetuned.search(query_embedding, k)
  scores = s[0]
  indices = indices[0]

  ranx_arr = []
  for i in range(len(indices)):
    id = data_idxs[indices[i]]
    score = float(scores[i])
    ranx_arr.append({'id' : id, 'score' : score})
  return ranx_arr

In [40]:
def search2(query, k):
  query_embedding = final.encode([query], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = index_finetuned.search(query_embedding, k)
  scores = s[0]
  indices = indices[0]
  ranx_arr = []
  for i in range(len(indices)):
    id = indices[i]
    score = float(scores[i])
    ranx_arr.append((score, id))
  return ranx_arr

In [41]:
def hybrid_search_test(query, k_b, k_f, alpha):
  bm_score = bm_25.get_scores(tokenize(query))
  bm_idx = np.argsort(bm_score)[::-1][:k_b]
  bm_score = scaler.fit_transform(bm_score.reshape(-1, 1))
  bm = {}
  for idx in bm_idx:
    id = text2[idx]
    bm[id] = bm_score[idx][0]
  query_embedding = final.encode([query], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = index_finetuned.search(query_embedding, k_f)
  s = scaler.fit_transform(s.reshape(-1, 1))
  faiss_r = {}
  for score_array, idx in zip(s, indices[0]):
      id = text2[idx]
      faiss_r[id] = float(score_array[0])
  indices = indices[0]

  total = set(bm.keys()) | set(faiss_r.keys())
  answer = []
  for id in total:
    bm_s = bm.get(id, 0)
    faiss_s = faiss_r.get(id, 0)
    score = alpha * (bm_s) + (1-alpha)*(faiss_s)
    answer.append({'id': id , 'score' : score})

  return answer

In [42]:
r_qrels = Qrels(test_qrels)
run2 = {}
for id, text in test_queries.items():
  results = search_test(text, 100)
  run2[id] = {i['id']:i['score'] for i in results}
ranx2_run = Run(run2)
metrics2 = evaluate(r_qrels, ranx2_run, ['mrr', 'ndcg@1', 'ndcg@5', 'ndcg@10', 'ndcg@20', 'recall@1', 'recall@5', 'recall@10', 'recall@100', 'precision@1', 'precision@5', 'precision@10'])

In [43]:
for metric, score in metrics2.items():
    print(metric, score)

mrr 0.010757777322963569
ndcg@1 0.01002865329512894
ndcg@5 0.010101218542578288
ndcg@10 0.010101218542578288
ndcg@20 0.010101218542578288
recall@1 0.008954154727793696
recall@5 0.010386819484240688
recall@10 0.010386819484240688
recall@100 0.011103151862464184
precision@1 0.01002865329512894
precision@5 0.0022922636103151865
precision@10 0.0011461318051575933


In [44]:
latency_list = []
for q in test_queries:
  start = time.time()
  query_embedding = final.encode([q], convert_to_numpy = True)
  faiss.normalize_L2(query_embedding)
  s, indices = index_finetuned.search(query_embedding, 10)
  finish = time.time()
  latency_list.append(finish - start)


In [45]:
np.percentile(np.array(latency_list), 50) #p50


np.float64(0.030701875686645508)

In [46]:
np.percentile(np.array(latency_list), 95) #p95

np.float64(0.03487062454223633)

In [47]:
np.percentile(np.array(latency_list), 99) #p99

np.float64(0.040891313552856426)

In [48]:
size_list = [1000, 5000, 10000, 25000]
scalability = {}
for i in size_list:
  samples = data_idxs[:i]
  sample_embed = embeddings_finetuned[:i]
  temp = faiss.IndexFlatIP(sample_embed.shape[1])
  faiss.normalize_L2(sample_embed)
  temp.add(sample_embed)
  test_queries2 = list(test_queries.values())[:100]
  latencies = []

  for query in test_queries2:
    start = time.time()
    query_emb = final.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_emb)
    scores, indices = temp.search(query_emb, 10)
    finish = time.time()
    latencies.append(finish - start)
  scalability[i] = np.median(latencies)

In [49]:
print(scalability)

{1000: np.float64(0.008778810501098633), 5000: np.float64(0.0092087984085083), 10000: np.float64(0.010113358497619629), 25000: np.float64(0.01416170597076416)}


In [50]:
import sys
index_size_mb = sys.getsizeof(faiss_idx) / (1024 ** 2)
embeddings_size_mb = embeddings_finetuned.nbytes / (1024 ** 2)

In [51]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

In [52]:
api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import time
import pickle
from beir.datasets.data_loader import GenericDataLoader
app = FastAPI(title='IEEERound2')
model = SentenceTransformer('/content/drive/MyDrive/finetuned')
index = faiss.read_index('finetuned_idx.index')
with open('data_idxs.pkl' , 'rb') as f:
  data_idxs = pickle.load(f)
with open('texts.pkl', 'rb') as f:
  text2 = pickle.load(f)
class Searching(BaseModel):
  query : str
  k : int = 10
@app.get('/')
def root():
  return {'documents' : len(data_idxs), 'status' : 'running'}
@app.post('/search')
def search(request: Searching):
  if not request.query.strip():
    raise HTTPException(status_code = 400, detail = 'Query is empty')
  if not (0 < request.k < 200) :
    raise HTTPException(status_code = 400, detail = 'top k has to be between 1 and 199')
  start = time.time()
  q_emb = model.encode([request.query], convert_to_numpy = True)
  faiss.normalize_L2(q_emb)
  s, idx = index.search(q_emb, request.k)
  results = []
  for rank, (i, j) in enumerate(zip(s[0], idx[0]), 1):
    results.append({'rank' : rank, 'id' : j, 'score' : i, 'text' : text2[j]})
  finish = time.time()
  l = finish - start
  return { 'query' : request.query, 'results' : results, 'latency' : l}'''
  #if __name__ == "__main__":
    #import uvicorn
    #uvicorn.run(app, host="0.0.0.0", port=8000)

In [53]:
with open('api.py', 'w') as f:
    f.write(api_code)

In [62]:
!nohup uvicorn api:app --reload --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

In [64]:
!pip install pyngrok
from pyngrok import ngrok
public = ngrok.connect(8000)
import requests
API = str(public)



In [65]:
print(public)

NgrokTunnel: "https://provable-usefully-wren.ngrok-free.dev" -> "http://localhost:8000"


In [66]:
!cat uvicorn.log

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [9319] using WatchFiles
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1914.67it/s, Materializing param=pooler.dense.weight]
INFO:     Started server process [9325]
INFO:     Waiting for application startup.
INFO:     Application startupINFO:     115.244.45.193:0 - "GET / HTTP/1.1" 200 OK
INFO:     115.244.45.193:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     115.244.45.193:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     115.244.45.193:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     115.244.45.193:0 - "POST /search HTTP/1.1" 500 Internal Server Error
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/fastapi/encoders.py", line 337, in jsonable_encoder
    data = dict(obj)
           ^^^^^^^^^
TypeError: 'numpy.int64' object is no

In [55]:
import ipywidgets as widgets
from IPython.display import display

In [56]:
query_box = widgets.Text(
    placeholder="Type query and press Enter",
    description="Query:",
    layout=widgets.Layout(width="70%")
)
search_button = widgets.Button(
    description="Search",
    button_style="primary"
)

output = widgets.Output()

In [57]:
def run_search(query):
  start = time.time()
  results = search2(query, 5)
  latency = time.time() - start
  with output:
    output.clear_output()
    print('latency : ' +str(latency))
    for rank, i in enumerate(results, 1):
      print(rank,text3[int(i[1])])
def on_submit(change):
    if change.value.strip():
        run_search(change.value)

query_box.on_submit(on_submit)
search_button.on_click(lambda b: run_search(query_box.value))
controls = widgets.VBox([
    query_box,
    search_button
])

display(controls, output)

Output()